# **Visualisations**

## Objectives
In this notebook, I will explore and communicate key trends in global video game sales to validate my hypotheses about what drives sales performance.  
I will begin with an overview of **global sales trends**, then create targeted visualisations to test each hypothesis — such as the impact of reviews, regional platform strength, and the effect of multi-platform and first-party releases.

## Inputs
I will use the **processed datasets** created during ETL and feature engineering:

- `video_game_sales_cleaned.csv` — base cleaned dataset with essential sales and metadata.
- `video_game_sales_critic_review.csv` — games with valid critic reviews.
- `video_game_sales_user_review.csv` — games with valid user reviews.
- `video_game_sales_merged.csv` — aggregated game-level dataset (platforms merged).
- `video_game_sales_first_party.csv` — dataset with engineered `is_first_party` and `is_multiplatform` columns.

## Outputs
- Visualisations that support each hypothesis:
  - **Global sales trends** over time.
  - Impact of **critic reviews** on sales.
  - Impact of **user reviews** on sales.
  - **Platform strength by region** (Nintendo, Sony, Microsoft, Other).
  - **Multi-platform vs single-platform** sales comparison.
  - **First-party vs third-party** sales comparison.
- Charts and summaries for the final Tableau dashboard.

## Approach
Each visualisation ties directly to one of my hypotheses and Kanban board user stories.  
I will start with a **Global Sales Trend** plot to establish market context, then move into deeper hypothesis testing using the processed datasets.

---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [ ]:
import os
current_dir = os.getcwd()
current_dir

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

Set up the data directories

In [ ]:
# Set the file path for the raw data
raw_data_dir = os.path.join(current_dir, 'data/raw')

# Set the file path for the processed data
processed_data_dir = os.path.join(current_dir, 'data/processed')

# Print file paths
print("Raw data directory:", raw_data_dir)
print("Processed data directory:", processed_data_dir)

# Imports

Import the necessary packages to perform the visualisation process.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from scipy.stats import linregress

# Load the csv files into dataframes

In [ ]:
df_master = pd.read_csv(os.path.join(processed_data_dir, 'video_game_sales_cleaned.csv'))
df_merged = pd.read_csv(os.path.join(processed_data_dir, 'video_game_sales_merged.csv'))
df_first_party = pd.read_csv(os.path.join(processed_data_dir, 'video_game_sales_first_party.csv'))
df_critic_reviews = pd.read_csv(os.path.join(processed_data_dir, 'video_game_sales_critic_review.csv'))
df_user_reviews = pd.read_csv(os.path.join(processed_data_dir, 'video_game_sales_user_review.csv'))

### You can view these dataframes [here](display_data.ipynb)

---

# Global Overview

### Global Sales Trends Over Time
- Plot total `Global_Sales` by `Year_of_Release`.
- Highlight peaks (e.g. Wii/DS era, PlayStation boom).

In [ ]:
# Aggregate global sales by year
global_sales_by_year = (
    df_master.groupby('Year_of_Release')['Global_Sales']
    .sum()
    .reset_index()
    .sort_values(by='Year_of_Release')
)

# filter out invalid years
global_sales_by_year = global_sales_by_year[global_sales_by_year['Year_of_Release'] > 0]

In [ ]:
# Plot global sales over time
plt.figure(figsize=(10, 5))
plt.plot(global_sales_by_year["Year_of_Release"], global_sales_by_year["Global_Sales"], marker="o")
plt.title("Global Video Game Sales Trends Over Time", fontsize=14)
plt.xlabel("Year of Release", fontsize=12)
plt.ylabel("Global Sales (Millions)", fontsize=12)
plt.grid(True, linestyle="--", alpha=0.5)
plt.show()

# Save the plot
plt.savefig(os.path.join(current_dir, 'images/visualisations/global_sales_trends.png'))

### Insights from Global Sales Trends

* **Steady Growth (1980s–1990s)**
  * Sales gradually rose as consoles like NES, SNES, and PlayStation became mainstream.

* **Peak Era (2000–2009)**
  - Sharp growth leading to a peak around **2008–2009**, driven by the Wii/DS boom and strong PS3/Xbox 360 releases.

* **Decline After 2009**
  - Sales drop steeply — likely due to digital distribution, mobile gaming, and fewer physical releases.

* **Recent Low Volumes (2015+)**
  - Physical sales remain low as the market shifts to digital downloads, subscriptions, and live-service games.

---
### Average Global Sales Over Time
- Plot average [(mean)](../README.md#core-statistical-concepts) `Global_Sales` by `Year_of_Release`.
- Highlight key shifts in the typical game’s performance

This is useful to understand how the sales per game changed over time — this avoids the distortion caused by years with unusually high numbers of releases and shows the market’s true health.

In [ ]:
# Calculate average global sales per game per year
avg_sales_by_year = (
    df_master.groupby('Year_of_Release')['Global_Sales']
    .mean()
    .reset_index()
    .sort_values('Year_of_Release')
)

# Filter out invalid years
avg_sales_by_year = avg_sales_by_year[avg_sales_by_year['Year_of_Release'] > 0]

In [ ]:
# Plot
plt.figure(figsize=(10,5))
plt.plot(avg_sales_by_year['Year_of_Release'], avg_sales_by_year['Global_Sales'], marker='o')
plt.title('Average Global Video Game Sales Per Game Over Time', fontsize=14)
plt.xlabel('Year of Release', fontsize=12)
plt.ylabel('Average Sales Per Game (Millions)', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

# Save the plot
plt.savefig(os.path.join(current_dir, 'images/visualisations/average_global_sales_trends.png'))

### Insights from Average Sales per Game by Year

* **Early spikes are unstable** — Pre-1995 values show sharp peaks and dips likely due to very small sample sizes (few games released per year, so one hit skews the average).
* **Long-term decline** — After the mid-90s, average sales per game dropped and stabilised around ~0.5–1M per title, showing a more saturated market and increased competition.
* **Hit-driven economy** — The steady low average despite total sales growth suggests the market relied on a few mega-hits (e.g., Wii Sports, GTA) while most games sold modestly.
* **Important check** — Outliers and low release counts in early years can distort averages

This plot complements total sales trends. It helps to avoid misleading conclusions (e.g., “sales peaked in 2008”) by showing that while total revenue was high, the *average game* didn’t necessarily sell better.

---

# **Hypothesis Testing**
---

## Hypothesis 1 — Critic Reviews Drive Sales
- **Data:** `video_game_sales_critic_review.csv`
- **Visual:** Scatter plot of `Critic_Score` vs `Global_Sales`  
  - Regression line  
  - Bubble size = `Critic_Count`
- **Validation:** Look for positive correlation & p-value < 0.05.

In [ ]:
plt.figure(figsize=(10,6))
sns.scatterplot(
    data=df_critic_reviews,
    x="Critic_Score",
    y="Global_Sales",
    size="Critic_Count",
    sizes=(20, 200),
    alpha=0.6
)
sns.regplot(
    data=df_critic_reviews,
    x="Critic_Score",
    y="Global_Sales",
    scatter=False,
    color="red",
    line_kws={'label':"Regression Line"}
)

plt.title("Critic Score vs Global Sales", fontsize=14)
plt.xlabel("Critic Score", fontsize=12)
plt.ylabel("Global Sales (Millions)", fontsize=12)
plt.legend()
plt.grid(alpha=0.3)
plt.show()

# Save the plot
plt.savefig(os.path.join(current_dir, 'images/visualisations/critic_score_vs_global_sales.png'))


In [ ]:
slope, intercept, r_value, p_value, std_err = linregress(
    df_critic_reviews["Critic_Score"], df_critic_reviews["Global_Sales"]
)

print(f"Correlation (r): {r_value:.3f}")
print(f"P-value: {p_value:.5f}")
print(f"Slope: {slope:.3f} (Global sales change per critic point)")


### Insights — Critic Scores vs Global Sales

The scatter plot shows a **positive but modest correlation** between critic scores and global sales (r = 0.245, p < 0.001). This indicates that games with higher critic scores **tend to sell more**, but the relationship is not very strong.

* **Statistical significance:** The p-value is effectively zero, so the relationship is highly significant.
* **Practical effect:** The slope suggests that for each additional point in critic score, global sales increase on average by ~0.03 million units. While small per point, the effect could accumulate (e.g., a 20-point score difference ≈ 0.6M extra sales).
* **Outliers:** Some blockbuster games with extreme sales overshadow most data points; filtering or using log scale helps to reveal the overall trend.
* **Critic count:** Larger bubbles (higher `Critic_Count`) cluster among higher-scoring and better-selling games, suggesting that games with more reviews are often also more commercially successful.

**Interpretation:**
Critic scores do appear to matter, but they explain only a small portion of sales variation. Other factors (franchise, marketing, platform, genre) likely have a bigger influence on commercial success.

---


Due to the fact that the above graph contains some obvious outliers.

I am going to filter these out to games selling less that 20M copies to create a more typical visualisation.

In [ ]:
# Filter out extreme outliers for readability (games selling >20M)
filtered = df_critic_reviews[df_critic_reviews["Global_Sales"] <= 20]

plt.figure(figsize=(10, 6))

# Bubble scatter plot using Seaborn
sns.scatterplot(
    data=filtered,
    x="Critic_Score",
    y="Global_Sales",
    size="Critic_Count",
    sizes=(20, 200),      # bubble size range
    alpha=0.5,
    legend="brief"
)

# Regression line (linear fit)
slope, intercept, r_value, p_value, std_err = linregress(
    filtered["Critic_Score"],
    filtered["Global_Sales"]
)
# x_vals = filtered["Critic_Score"]
sns.lineplot(
    x=filtered["Critic_Score"],
    y=intercept + slope*filtered["Critic_Score"],
    color="red",
    linewidth=2,
    label="Regression line"
)

# Plot formatting
plt.xlabel("Critic Score", fontsize=12)
plt.ylabel("Global Sales (Millions)", fontsize=12)
plt.title("Critic Score vs Global Sales (Excluding Outliers >20M)", fontsize=14)
plt.grid(True, linestyle="--", alpha=0.3)
plt.legend()
plt.show()

# Save the plot
plt.savefig(os.path.join(current_dir, 'images/visualisations/critic_score_vs_global_sales_filtered.png'))


In [ ]:
# Display correlation stats
print(f"Correlation (r): {r_value:.3f}")
print(f"P-value: {p_value:.5f}")
print(f"Slope: {slope:.3f} (Global sales change per critic point)")

### Insights — Critic Reviews Drive Sales (Filtered - <20M Global Sales)

The scatter plot shows a clearer positive correlation between critic scores and global sales once extreme outliers are removed (r = 0.331, p < 0.001). This suggests that critic reviews have a moderate influence on sales.

* **Statistical significance:** The p-value is effectively zero, confirming the relationship is highly significant.
* **Practical effect:** The slope indicates each additional critic point adds about **0.03M sales** on average. A 20-point difference could equate to ~0.6M extra sales.
* **Outliers:** Removing blockbuster hits (>20M sales) improves readability and highlights the general trend.
* **Critic count:** Larger bubbles (higher Critic_Count) still cluster toward higher scores and better sales, showing that widely reviewed, well-rated games tend to sell more.

**Interpretation:** Critic scores appear to have a stronger and clearer link to sales after filtering extreme hits. However, the relationship is still moderate — other commercial drivers such as brand recognition, marketing, and platform likely play major roles.

---

## Overall Conclusion

### **Higher critic review scores positively influence sales.**

### Statistical Validation

* **Correlation (all data):** r = **0.245** (positive but modest)
* **Correlation (filtered <20M):** r = **0.331** (stronger once extreme hits removed)
* **P-value:** <0.001 in both cases → **highly significant relationship**
* **Slope:** ≈ **0.03M extra sales per 1-point critic score increase**
  → 20-point difference ≈ +0.6M global sales

### Interpretation

* The hypothesis is **statistically supported** — critic scores do have a positive and significant relationship with global sales.
* **However**, the effect size is modest, meaning critic reviews influence sales but don’t fully explain commercial success. Other factors like brand/franchise power, marketing budgets, and platform impact likely play a larger role.
* Filtering extreme outliers made the trend clearer and improved correlation, indicating blockbuster games can distort the overall picture.

**Validation Decision:**
* Hypothesis **ACCEPTED** — critic reviews significantly influence sales, but the impact is moderate rather than dominant.

---

## Hypothesis 2 — User Reviews Drive Sales

* **Data:** `video_game_sales_user_review.csv`
* **Visual:** Scatter plot of `User_Score` vs `Global_Sales`

  * Regression line
  * Bubble size = `User_Count`
* **Validation:** Look for a positive correlation (p-value < 0.05) between user score and global sales.

---

## Hypothesis 3 — Platform Strength by Region

* **Data:** `video_game_sales_cleaned.csv`
* **Visual:** Grouped bar charts of average `JP_Sales`, `NA_Sales`, and `EU_Sales` by platform vendor (Nintendo, Sony, Microsoft, Other).
* **Validation:** Use ANOVA and post-hoc Tukey tests to confirm significant differences between vendors in each region (expect Nintendo highest in Japan, Sony/Microsoft leading in NA and EU).


---

## Hypothesis 4 — Multi-Platform Advantage

* **Data:** `video_game_sales_merged.csv`
* **Visual:** Boxplot or violin plot comparing `Global_Sales` for `is_multiplatform = True` vs `False`.
* **Validation:** Test for a statistically significant difference in sales using Mann–Whitney U or t-test (expect multi-platform games to sell more globally).

---

## Hypothesis 5 — First-Party Advantage

* **Data:** `video_game_sales_first_party.csv`
* **Visual:** Bar charts comparing mean `Global_Sales` for `is_first_party = True` vs `False`, grouped by platform vendor (Nintendo, Sony, Microsoft).
* **Validation:** Use t-test or Mann–Whitney U to check if first-party games significantly outperform third-party titles on their respective platforms.

---

## Dashboard Preparation
- Export any summary tables or aggregated CSVs that will feed into Tableau.
- Note which visuals or KPIs will be highlighted (e.g. average sales per platform, first-party vs third-party performance, critic vs user review impact).

---

## Insights & Next Steps
- Summarise the main findings from each hypothesis test.
- Highlight actionable insights (e.g. “Critic reviews show stronger correlation than user reviews”; “First-party Nintendo titles outperform third-party significantly in Japan”).
- Note next steps (interactive Tableau dashboard build).

---